# Terminology
__Automatic Differentiation (autodiff)__ a general way of taking a program which computes a value, and automatically constructing a procedure for computing derivatives of that value. 

__Backpropagation__ the special case of autodiff applied to nn 

__Autograd__ the name of a particular autodiff library

# Finite Differences
Note that by the definition of derivative 
$$\partial_{x_i}f(x_1,...,x_N) = \lim_{h\rightarrow 0}\frac{f(x_1,...,x_i+h, ..., x_N)-f(x_1,...,x_i, ..., x_N)}{h}$$ or 
$$\partial_{x_i}f(x_1,...,x_N) = \lim_{h\rightarrow 0}\frac{f(x_1,...,x_i+h, ..., x_N)-f(x_1,...,x_i-h, ..., x_N)}{2h}$$

Finite differences are expensive, since you need to do a forward pass for each derivative. Also, this may include huge numerical error. 

However, since it directly comes from definition, it is often used for testing. 

# Autodiff
An autodiff system will convert the program into a sequence of primitive ops which have specified routines for computing derivatives. 

## Computational Graph
The `Node` class represents a node of the computation graph, with attributes
 - value: the actual value computed on a particular set of inputs
 - function: the primitive operation defining the node
 - args and kwargs: the arguments the op was called with
 - parents: the parent `Node`

## Vector-Jacobian Products
The __Jacobian__ is the matrix pf partial derivatives
$$J = \frac{\partial \vec y}{\partial \vec x} = 
\begin{bmatrix}
\partial_{x_1}y_1&\cdots&\partial_{x_n}y_1\\
\vdots&\ddots&\vdots\\
\partial_{x_1}y_m&\cdots&\partial_{x_n}y_m
\end{bmatrix}$$
Then, the backprop equation can be written as a VJP 
$$\bar{x_j} = \sum_i \bar{y_i}\frac{\partial y_i}{\partial x_j}, \bar x = \bar y^T J(\text{row vector}), \bar x = J^T\bar y(\text{col vector})$$

For each primitive op, we must specify VJPs for each of its arguments. This is a function which takes in the output gradient (i.e. $\bar y$). the answer ($y$), and the arguments ($x$), and returns the input gradient ($\bar x$). 

## Backprop as Message Passing
Each node receives a bunch of messages from its children, which it aggregates to get its error signal. It then passes messages to its parents. Each of these messages is a VJP.   
This formulation provides modularity: each node needs to know how to compute its outgoing messages, i.e., the VJPs corresponding to each of its parents (arguments to the function). The implementation of $z$ doesn't need to know where $\bar z$ came from. 